# Project Luther: Predicting the Market Value of NBA Players

Name: Paul Lim

Date: 04/28/2017

In [ ]:
# Relevant libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import time
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import cross_val_score, train_test_split, KFold, GridSearchCV
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn import pipeline, feature_selection, decomposition
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import RFECV
import logging


import warnings
warnings.filterwarnings('ignore')

sns.set_style("white")
sns.set_style('ticks')
sns.set_style({'xtick.direction': u'in', 'ytick.direction': u'in'})
sns.set_style({'legend.frameon': True})

%matplotlib inline

## List of Classes/Functions

In [ ]:
def bball_get_col(table, col_tag='th'):
    '''
    DESCRIPTION:
        - Generate the list of column names from the HTML table.
    INPUT:
        - table is an HTML table found on basketball-reference.com
        - col_tag is set to th as default. 
    OUTPUT:
        - list of column names is outputted
    '''
    col_loc = table.find('tr')
    cols = col_loc.find_all(col_tag)

    cols_list = []
    for i in range(len(cols)):
        temp_col = cols[i].get_text()
        cols_list.append(temp_col)
        
    return cols_list

In [ ]:
def bball_get_data(table):
    '''
    DESCRIPTION:
        - Aggregate the data from the HTML table.
    INPUT:
        - table is an HTML table found on basketball-reference.com
    OUTPUT:
        - data from the HTML table is outputted as a list of lists.
    '''
    all_rows = table.find_all('tr')[1:]

    all_data = []

    for i in range(len(all_rows)):
        row = all_rows[i].find_all('th')
        add_row = all_rows[i].find_all('td')
        row.extend(add_row)
        data = []
        for j in range(len(row)):
            datapoint = row[j].get_text()
            data.append(datapoint)
        all_data.append(data)
        
    return all_data

In [ ]:
def bball_scrape_data(url_template, start_year, end_year, delay=5):
    '''
    DESCRIPTION:
        - Creates a pandas dataframe object from the data scraped from basketball-reference.com.
    INPUT:
        - url_template is the url that contains the data table that will be scraped.
        - start_year and end_year give the range of years to scrape.
        - delay is the number of seconds in-between moving on to the next url.
    OUTPUT:
        - data from the HTML table is outputted as a dataframe.
    '''
    df = pd.DataFrame()
    
    if start_year > end_year:
        return print('Enter in valid end year.')
    else: 
        for year in range(start_year, end_year+1):
            try:
                url = url_template.format(year=year)
                link = requests.get(url)
            except:
                print(url)
                print('Check to make sure the URL is correct!')
            
            page = link.text

            soup = BeautifulSoup(page, 'lxml')
            table = soup.find('table')

            col_list = bball_get_col(table)
            all_data = bball_get_data(table)
            
            if not col_list or not all_data:
                print(url)
                print('Webpage may be empty.')
                pass
            elif len(col_list) != len(all_data[0]):
                print('Column List: \n', col_list)
                print('Data Row: \n', all_data)
                return print('Make sure the length of columns and data are consistent!')
            else:
                temp_df = pd.DataFrame(all_data, columns=col_list)
                temp_df = temp_df.assign(Yr = year)
                df = df.append(temp_df)
            time.sleep(delay)

    return df

In [ ]:
def sal_get_col(table_sal):
    '''
    DESCRIPTION:
        - Generate the column names for the salary table.
    INPUT:
        - table_sal is the HTML table from ESPN.com.
    OUTPUT:
        - list of column names is outputted.
    '''
    col_loc_sal = table_sal.find('tr')
    cols_sal = col_loc_sal.find_all('td')

    cols_list_sal = []
    for i in range(len(cols_sal)):
        temp_col_sal = cols_sal[i].get_text()
        cols_list_sal.append(temp_col_sal)
    return cols_list_sal

In [ ]:
def sal_get_data(table_sal):
    '''
    DESCRIPTION:
        - Aggregate the data from the salary table.
    INPUT:
        - table_sal is the HTML table from ESPN.com.
    OUTPUT:
        - data from the salary table is outputted as a list of lists.
    '''
    all_rows_sal = table_sal.find_all('tr', class_ = ['evenrow', 'oddrow'])

    all_data_sal = []

    for i in range(len(all_rows_sal)):
        row_sal = all_rows_sal[i].find_all('td')
        data_sal = []
        for j in range(len(row_sal)):
            datapoint_sal = row_sal[j].get_text()
            data_sal.append(datapoint_sal)
        all_data_sal.append(data_sal)
        
    return all_data_sal

In [ ]:
def sal_scrape_data(url_template, start_year, end_year, start_page, end_page, delay=5):
     '''
    DESCRIPTION:
        - Creates a pandas dataframe object from the salary table on ESPN.com.
    INPUT:
        - url_template is the url that contains the data table that will be scraped.
        - start_year and end_year give the range of years to scrape.
        - start_page and end_page give the number of pages available for each year.
        - delay is the number of seconds in-between moving on to the next url.
    OUTPUT:
        - data from the HTML table is outputted as a dataframe.
    '''
    df = pd.DataFrame()
    
    if start_year > end_year:
        return print('Enter in valid end year.')
    elif start_page > end_page:
        return pring('Enter in a valid end page.')
    else: 
        for year in range(start_year, end_year+1):
            for page in range(start_page, end_page+1):
                try:
                    url = url_template.format(year=year, page=page)
                    link = requests.get(url)
                except:
                    print(url)
                    print('Check to make sure the URL is correct!')
                
                page = link.text

                soup = BeautifulSoup(page, 'lxml')
                table = soup.find('table')

                col_list = sal_get_col(table)
                all_data = sal_get_data(table)
                
                if not col_list or not all_data:
                    print(url)
                    print('Webpage may be empty.')
                    pass
                elif len(col_list) != len(all_data[0]):
                    print('Column List: \n', col_list)
                    print('Data Row: \n', all_data)
                    return print('Make sure the length of columns and data are consistent!')
                else:
                    temp_df = pd.DataFrame(all_data, columns=col_list)
                    temp_df = temp_df.assign(Yr = year)
                    df = df.append(temp_df)
                time.sleep(delay)
                
    return df

In [ ]:
def clean_player_data(df_list):
    '''
    DESCRIPTION:
        - Clean the messy data that was scraped from basketball-reference.com.
    INPUT:
        - df_list is a list of dataframes containing scraped data.
    OUTPUT:
        - clean_df_list is a list of clean dataframes.
    '''
    clean_df_list = []
    for df in df_list:
        # Remove the Unnamed and Rk columns
        if 'Unnamed: 0' in df.columns:
            df = df.drop('Unnamed', axis=1)
        if 'Rk' in df.columns:
            df = df.drop('Rk', axis=1)
        
        # Remove the rows there the Tm column value is 'Tm'.
        df = df[df['Tm'] != 'Tm']
        
        # Remove characters such as asteriks from player names.
        df['Player'] = df['Player'].map(lambda x: re.sub(r"\*", '', x))
        
        # Some players played on multiple teams in the same year due to trades.
        # These players will have their stats averaged to one value.
        df = df.sort(columns=['Player','Yr','Pos','Tm'])
        df = df.apply(pd.to_numeric, errors='ignore')
        df = df.groupby(['Player','Yr','Pos','Tm']).mean().reset_index()
        
        clean_df_list.append(df)
    return clean_df_list

In [ ]:
def clean_salary_data(df):
    '''
    DESCRIPTION:
        - Clean the messy data that was scraped from ESPN.com.
    INPUT:
        - df is a dataframe containing scraped data.
    OUTPUT:
        - clean_df is a clean dataframe.
    '''
    # Remove the Unnamed and Rk columns
    if 'Unnamed: 0' in df.columns:
        df = df.drop('Unnamed', axis=1)
    if 'Rk' in df.columns:
        df = df.drop('Rk', axis=1)
        
    # Edit the Name column so that only the names are left.
    df['NAME'] = df['NAME'].map(lambda x: re.sub(r",\s.+", '', x))
    
    # Edit the Salary column to an integer value.
    df['SALARY'] = df['SALARY'].map(lambda x: re.sub(r"\$", '', x))
    df['SALARY'] = df['SALARY'].map(lambda x: re.sub(r",", '', x))
    df['SALARY'] = df['SALARY'].map(lambda x: int(x))
    
    # Adjust the Salary column to account for inflation.
    # Average rate of inflation between 2000 and 2016 was 2.15%
    df['yr_diff'] = df['Yr'].map(lambda x: 2017 - x)
    df['salary_adj'] = df.apply(lambda x: x['SALARY']*(1 + 0.0215)**x['yr_diff'], axis=1).map(int)
    
    # Ultimately the salary dataframe will only need the Name, Year, Salary, and Adjusted Salary columns. 
    # The Name and Yr columns will be used as the key when joining the two databases.
    df = df.loc[:,['NAME','Yr', 'SALARY', 'salary_adj']]
    
    return df

In [1]:
def merge_df(df_tot, df_adv, df_sal):
    '''
    DESCRIPTION:
        - Merge the total stat, advanced stat, and salary dataframes.
    INPUT:
        - df_tot is the total stat dataframe.
        - df_adv is the advanced stat dataframe.
        - df_sal is the salary dataframe.
    OUTPUT:
        - df_merge is the merged dataframe.
    '''
    # Since df_tot and df_adv have duplicate columns these columns will be excluded when merged.
    col_to_use = df_tot.columns - df_adv.columns
    df_merge = df_adv.merge(df_tot[col_to_use], how='outer', left_index=True, right_index=True)
    
    # The salary dataframe and the combined dataframe from above should be joined on the player name and year.
    df_merge = df_sal.merge(df_merge, how='inner', left_on=['NAME', 'Yr'], right_on=['Player', 'Yr'])
    df_merge = df_merge.drop('Player', axis=1)
    
    # Replace null values with 0
    df_merge['3P%'] = df_merge.loc[:,'3P%'].fillna(0)
    df_merge['2P%'] = df_merge.loc[:, '2P%'].fillna(0)
    df_merge['FT%'] = df_merge.loc[:, 'FT%'].fillna(0)
    df_merge['3PAr'] = df_merge.loc[:, '3PAr'].fillna(0)
    df_merge['FTr'] = df_merge.loc[:, 'FTr'].fillna(0)
    df_merge['FG%'] = df_merge.loc[:, 'FG%'].fillna(0)
    df_merge['eFG%'] = df_merge.loc[:, 'eFG%'].fillna(0)
    
    # Create dummy variables for position.
    pos_dict = {'C': 'C', 'PF': 'PF','SF': 'SF','PG':'PG','SG':'SG','C-PF': 'C','SF-SG':'SF','PG-SG':'PG',
               'PG-SF':'PG','SG-SF':'SG','PF-C':'PF','PF-SF':'PF','SG-PG':'SG','SG-PF':'SG','SF-PF':'SF'}
    df_merge['Pos'] = df_merge['Pos'].map(pos_dict)
    df_merge = pd.get_dummies(df_merge, columns=['Pos'])
    
    # There are columns with all null values. These columns will be dropped.
    df_merge = df_merge.dropna(axis=1, how='all')
    df_merge = df_merge.sample(frac=1)
    
    return df_merge

In [ ]:
def square_features(df, col_list):
    '''
    DESCRIPTION:
        - Square certain features to create new features.
    INPUT:
        - df is the input dataframe.
        - col_list is the list of columns that need to be squared.
    OUTPUT:
        - df is the output dataframe with new squared features.
    '''
    for col in col_list:
        new_col_name = col + '_sq'
        df[new_col_name] = df[col].apply(lambda x: x**2)
    return df

In [ ]:
def cube_features(df, col_list):
    '''
    DESCRIPTION:
        - Cube certain features to create new features.
    INPUT:
        - df is the input dataframe.
        - col_list is the list of columns that need to be cubed.
    OUTPUT:
        - df is the output dataframe with new cubed features.
    '''
    for col in col_list:
        new_col_name = col + '_cube'
        df[new_col_name] = df[col].apply(lambda x: x**3)
    return df

In [ ]:
def rank_features(df, groupby_list, rank_list):
    '''
    DESCRIPTION:
        - Rank players on each team for certain stats to create new features.
    INPUT:
        - df is the input dataframe.
        - groupby_list is the list of features to group the input dataframe by.
        - rank_list is the list of columns that will be used to rank players.
    OUTPUT:
        - df is the output dataframe with new cubed features.
    '''
    df_rank = df.groupby(groupby_list).rank(ascending=True)
    new_col_list = []
    for col in rank_list:
        new_col_name = col + '_rk'
        df_rank[new_col_name] = df_rank[col]
        new_col_list.append(new_col_name)
    return df_rank[new_col_list]

In [2]:
def feature_engineering(df, ratios=True, rank=True):
    '''
    DESCRIPTION:
        - New features were created after preliminary data analysis. These new features may or may not be used.
    INPUT:
        - df is the input dataframe.
        - ratios denotes if ratio of stats should be created.
        - rank denotes if players on teams should be ranked by certain stats.
    OUTPUT:
        - df is the output dataframe with new squared features.
    '''
    # Square PER, AGE, GS, G, USG%
    df = square_features(df, ['PER','Age','GS','G','USG%'])

    # Cube Age, G, USG%
    df = cube_features(df, ['Age','G','USG%'])
    
    if ratios:
        # Create ratios: PTS/TOV, AST/TOV, STL/TOV, BLK/TRB, STL/AST
        df['PTS/TOV'] = (df['PTS']/df['TOV']).replace([np.inf, -np.inf], np.nan).fillna(value=0)
        df['AST/TOV'] = (df['AST']/df['TOV']).replace([np.inf, -np.inf], np.nan).fillna(value=0)
        df['STL/TOV'] = (df['STL']/df['TOV']).replace([np.inf, -np.inf], np.nan).fillna(value=0)
        df['BLK/TRB'] = (df['BLK']/df['TRB']).replace([np.inf, -np.inf], np.nan).fillna(value=0)
        df['STL/AST'] = (df['STL']/df['AST']).replace([np.inf, -np.inf], np.nan).fillna(value=0)

        # Create ratios: WS/PER, OWS/PER, WS/OWS, BPM/OWS, BPM/WSper48
        df['WS/PER'] = (df['WS']/df['PER']).replace([np.inf, -np.inf], np.nan).fillna(value=0)
        df['OWS/PER'] = (df['OWS']/df['PER']).replace([np.inf, -np.inf], np.nan).fillna(value=0)
        df['WS/OWS'] = (df['WS']/df['OWS']).replace([np.inf, -np.inf], np.nan).fillna(value=0)
        df['BPM/OWS'] = (df['BPM']/df['OWS']).replace([np.inf, -np.inf], np.nan).fillna(value=0)
        df['BPM/WSper48'] = (df['BPM']/df['WS/48']).replace([np.inf, -np.inf], np.nan).fillna(value=0)
        
    if rank:
        # Create features that will rank players in each team and each year.
        rank_col_list = ['PTS','AST','TRB','BLK','STL','TOV','MP','WS','PER']
        group_col_list = ['Yr','Tm']
    
        df_rank = rank_features(df, group_col_list, rank_col_list)
        df = df.merge(df_rank, left_index=True, right_index=True)


    # Change target feature to log(salary_adj)
    df['salary_adj_ln'] = df['salary_adj'].apply(np.log)
    
    return df

In [ ]:
def run_pipelines(model_dict, X, y, feature_list, param_dict):
    '''
    Runs through a pipeline for each type of model.
    feature_list = list of tuples.
    param_dict = a nested dictionary that contains the hyper parameters that need to be tuned.
    '''
    logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        
    grid_dict = {}
    
    for name, model in sorted(model_dict.items()):
        if feature_list:
            combined_features = pipeline.FeatureUnion(feature_list)

            steps = [
                ('features', combined_features),
                ('feature_selection', feature_selection.SelectFromModel(linear_model.Lasso(alpha=2))),
                ('model', model)
                ]
        else:
            steps = [
                ('feature_selection', feature_selection.SelectFromModel(linear_model.Lasso(alpha=2))),
                ('model', model)
                ]
        
        regression_pipeline = pipeline.Pipeline(steps)
        if name in param_dict:
            parameters = param_dict[name]
        else:
            return print('Incorrect parameters in the parameter dictionary.')
            
        grid_dict[name] = GridSearchCV(regression_pipeline, parameters, n_jobs=3, verbose=1)
        train_fit = grid_dict[name].fit(X_train, y_train)
        print('Model: ', name)
        print('Best Score: %0.3f' % train_fit.best_score_)
        print('Optimal Parameters: ', train_fit.best_params_)
         
    return grid_dict

## Web Scrape

In [ ]:
bball_ref_url = 'http://www.basketball-reference.com/leagues/NBA_{year}_advanced.html'
bball_tot_url = 'http://www.basketball-reference.com/leagues/NBA_{year}_totals.html'
bball_per_url = 'http://www.basketball-reference.com/leagues/NBA_{year}_per_game.html'
salary_url = 'http://www.espn.com/nba/salaries/_/year/{year}/page/{page}/seasontype/3'

start_year = 2000
end_year = 2017
start_page = 1
end_page = 4

df_bball = bball_scrape_data(bball_ref_url, start_year, end_year)
df_bball_tot = bball_scrape_data(bball_tot_url, start_year, end_year)
df_bball_per = bball_scrape_data(bball_per_url, start_year, end_year)
df_sal = sal_scrape_data(salary_url, start_year, end_year, start_page, end_page)

## Cleaning Data

In [ ]:
clean_df_list = clean_player_data([df_bball, df_bball_tot])
df_salary = clean_salary_data(df_sal)

## Merging Dataframes

Join dataframes on a key formed by the year and name of the player.
Inner join will be used since the players that don't have salary information will be excluded.

In [ ]:
df_merge = merge_df(clean_df_list[1], clean_df_list[0], df_sal)

# Model Building

In [ ]:
df_merge = feature_engineering(df_merge)

X_cols = ['Age', 'G',
       'MP', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%',
       'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM',
       'DBPM', 'BPM', 'VORP', '2P', '2P%', '2PA', '3P', '3P%', '3PA', 'AST',
       'BLK', 'DRB', 'FG', 'FG%', 'FGA', 'FT', 'FT%', 'FTA', 'GS', 'ORB', 'PF',
       'PTS', 'STL', 'TOV', 'TRB', 'eFG%', 'Pos_C', 'Pos_PF', 'Pos_PG',
       'Pos_SF', 'Pos_SG', 'PER_sq', 'Age_sq', 'GS_sq', 'G_sq', 'USG%_sq',
       'Age_cube', 'G_cube', 'USG%_cube', 'PTS/TOV', 'AST/TOV', 'STL/TOV',
       'BLK/TRB', 'STL/AST', 'WS/PER', 'OWS/PER', 'WS/OWS', 'BPM/OWS',
       'BPM/WSper48', 'PTS_rk', 'AST_rk', 'TRB_rk', 'BLK_rk', 'STL_rk',
       'TOV_rk', 'MP_rk','WS_rk','PER_rk']

# Create the X and y matrices. 
# Also, set apart 15% of the data for a holdout set.
holdout = df_merge.iloc[0:403, :]

df_test = df_merge.iloc[403:,:]

X = df_test.iloc[403:,:]
X = X.loc[:, X_cols]
y = df_test.iloc[403:,:]
y = y.loc[:, 'salary_adj']

X_arr = X.as_matrix()
y_arr = y.as_matrix()

## Use pipelines to choose the best model with tuned hyper parameters

In [ ]:
pipe_dict = {}
pipe_dict['gradient_boost'] = GradientBoostingRegressor()
pipe_dict['random_forest'] = RandomForestRegressor()
pipe_dict['lin_reg'] = linear_model.LinearRegression()
pipe_dict['lasso'] = linear_model.Lasso()
pipe_dict['ridge'] = linear_model.Ridge()


param_dict = {}
param_dict['gradient_boost'] = {'model__n_estimators': [10,100,300], 
                              'model__max_depth': [2,3,4,5], 
                              'model__min_samples_split': [5,10,20,40],
                              'model__max_features': ['sqrt', 'auto']}
param_dict['random_forest'] = {'model__n_estimators': [10,100,300],
                             'model__max_depth': [2,3,4,5], 
                             'model__min_samples_split': [5,10,20,40],
                             'model__max_features': ['sqrt', 'auto']}
param_dict['lin_reg'] = {'model__fit_intercept': [True]}
param_dict['lasso'] = {'model__alpha': [.01,.1,1,10,100]}
param_dict['ridge'] = {'model__alpha' : [.01,.1,1,10,100]}
param_dict['elastic'] = {'model__alpha': [.01,.1,1,10,100]}

In [ ]:
grid = run_pipelines(pipe_dict, X, y, feature_list=None, param_dict)